Created Date : December 2021 <br>
Written By : [Deepa Goyal](https://www.linkedin.com/in/deepag/)

# Updating Books Info on Notion using Google Books API

https://developers.notion.com/

https://towardsdatascience.com/productivity-tracking-with-the-notion-api-and-python-f5f866fe11d8

Notion provides a very thorough [reading list template](https://www.notion.so/templates/notion-reading-list) that once i started using, I go hooked to. I have been reviewing my reading list. As the list of my finished books grew, I kept adding more to my list. I also enjoy using this template to take notes from the books that I’m reading. Having it all in one place has allowed me to revisit them more often than I used to.

<img src="https://i.ibb.co/F3PFTdw/Brand-building-blog-images.png" alt="Brand-building-blog-images" border="0">

The view on Notion’s template gallery looks neat with images and titles. It has the classic iTunes library look that I find just perfect.

<img src="https://i.ibb.co/1vdbbJP/screencapture-notion-so-Reading-List-14918425560f4ec5b303314a29eb53e1-2021-12-30-11-51-48.png" alt="screencapture-notion-so-Reading-List-14918425560f4ec5b303314a29eb53e1-2021-12-30-11-51-48" border="0">

But then I looked at my Reading List, I didn’t have any images unless I was manually uploading them. Ugh!

<a href="https://ibb.co/KmfHTnW"><img src="https://i.ibb.co/qxcwT6p/Screen-Shot-2021-12-30-at-12-01-16-PM.png" alt="Screen-Shot-2021-12-30-at-12-01-16-PM" border="0"></a>

To get to the aesthetically pleasing view I would have to do much more **data entry work** which isn’t something I’m excited about. So I decided to write a python script to do it for me.

As I started looking into it, I found out that [Goodreads API](https://www.goodreads.com/api) has been deprecated as of December 2020, which is a shame because I have seen a few good projects people have done using those. The only other free books API I could find was Google books API.

Here in project you would see the code that ultimately made my Reading list look like a Book shelf.

<img src="https://i.ibb.co/dkqj9Nk/Screen-Shot-2021-12-30-at-12-18-25-PM.png" alt="Screen-Shot-2021-12-30-at-12-18-25-PM" border="0">

Google’s API was fairly straight forward to use. But I was quite surprised that of all the data it provides, it doesn’t have a link to the google books page for the given book.

## Getting started with the Notion API

1. Finding the Database ID and Page ID - This is pretty deep in the page, but very important to get started. The database ID is 32 character alphanumeric part of your URL to the database.

<img src="https://i.ibb.co/3zjKJ3Z/Screen-Shot-2021-12-30-at-12-30-09-PM.png" alt="Screen-Shot-2021-12-30-at-12-30-09-PM" border="0">

1. Sharing the page you want your App to access with the App - Notion API’s get started page has a good guide with gifs showing how to create an App, share your page with the app to get started. I somehow missed this page at first and found some posts on stack overflow where other people had too. So don’t forget to share your page with the app.
2. Query database vs Retrive database - I struggled with Retrieve database endpoint trying to parse the data but it was not complete and parsing it was too many steps. Query database endpoint was much more useful for and intuitive.
3. Dealing with datatypes of the properties - The data is nested too deep and to update it, you have to build the vocabulary for it.

I had tremendous fun doing this project. The only thing I wish for was if Notion provided a way to import this Jupyter notebook into a notion page so that I could do all my work here and publish it as a blog using notion. When i tried to import this page as HTML, i had to do a lot of manual edits in notion, when i imported it as markdown, code wasn't imported or formatted properly.

In [1]:
import pandas as pd
import requests
import json
import math

### Google Books Api search based on search term

In [2]:
# google search for the book using google books API
def google_book_search(search_terms):
    url = 'https://www.googleapis.com/books/v1/volumes?q='
    response = requests.get(url+search_terms, verify=False)
    # load data using Python JSON module
    r = response.content
    data = json.loads(r)
    # Normalizing data
    df = pd.json_normalize(data, record_path=['items'])
    first_row = df.iloc[0]
    first_row_df = df.iloc[:1]
    first_row_df['search_term'] = search_terms
    return first_row_df

### Parameters

In [10]:
secret_key = 'secret_9fpBgL5ksvA2BaSAHEiW6GFgF2nDCTBlVw2C1n5mHlU'
database_id = '1b71653e363148e0b2ba399638473ced'

### Query database

In [11]:
def query_databases(secret_key, database_id):
    url = "https://api.notion.com/v1/databases/"+database_id+'/query'
    # print(url)

    payload = {'id': database_id}
    headers = {
        'Notion-Version': '2021-05-13',
        # "Content-Type": "application/json",
        'Authorization': 'Bearer '+secret_key
    }

    response = requests.request(
        "POST", url, headers=headers, data=payload, verify=False)
    print(f"The response code is {response.status_code}")
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    else:
        return response.json()

### Get data from Reading List

In [12]:
res = query_databases(secret_key, database_id)
res

/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


The response code is 200


{'object': 'list',
 'results': [{'object': 'page',
   'id': '8ae970fc-c503-4b6c-a551-eee7e434b2b4',
   'created_time': '2024-08-12T17:00:00.000Z',
   'last_edited_time': '2024-08-12T17:00:00.000Z',
   'created_by': {'object': 'user',
    'id': '074559e7-8c13-4b7a-899e-4012f06d6320'},
   'last_edited_by': {'object': 'user',
    'id': '074559e7-8c13-4b7a-899e-4012f06d6320'},
   'cover': None,
   'icon': {'type': 'external',
    'external': {'url': 'https://www.notion.so/icons/book-closed_gray.svg'}},
   'parent': {'type': 'database_id',
    'database_id': '1b71653e-3631-48e0-b2ba-399638473ced'},
   'archived': False,
   'in_trash': False,
   'properties': {'Category': {'id': '/7eo',
     'type': 'select',
     'select': {'id': 'r\\pO', 'name': 'Fiction', 'color': 'default'}},
    'Summary': {'id': '?\\25', 'type': 'rich_text', 'rich_text': []},
    'Current page': {'id': 'QK`M', 'type': 'number', 'number': 20},
    'Total pages': {'id': '\\?}w', 'type': 'number', 'number': 920},
    'Dat

### Results has the data we need

In [13]:
# dict_keys(['object', 'results', 'next_cursor', 'has_more'])
res.get('results')

[{'object': 'page',
  'id': '8ae970fc-c503-4b6c-a551-eee7e434b2b4',
  'created_time': '2024-08-12T17:00:00.000Z',
  'last_edited_time': '2024-08-12T17:00:00.000Z',
  'created_by': {'object': 'user',
   'id': '074559e7-8c13-4b7a-899e-4012f06d6320'},
  'last_edited_by': {'object': 'user',
   'id': '074559e7-8c13-4b7a-899e-4012f06d6320'},
  'cover': None,
  'icon': {'type': 'external',
   'external': {'url': 'https://www.notion.so/icons/book-closed_gray.svg'}},
  'parent': {'type': 'database_id',
   'database_id': '1b71653e-3631-48e0-b2ba-399638473ced'},
  'archived': False,
  'in_trash': False,
  'properties': {'Category': {'id': '/7eo',
    'type': 'select',
    'select': {'id': 'r\\pO', 'name': 'Fiction', 'color': 'default'}},
   'Summary': {'id': '?\\25', 'type': 'rich_text', 'rich_text': []},
   'Current page': {'id': 'QK`M', 'type': 'number', 'number': 20},
   'Total pages': {'id': '\\?}w', 'type': 'number', 'number': 920},
   'Date started': {'id': 'evL[',
    'type': 'date',
    '

### Get property data

In [14]:
# results are in list format, this has all the items from the database, validate the number of items to make sure all items were read successfully
results = res.get('results')
df = pd.DataFrame([])
print(type(df))
print(len(list(results)))

<class 'pandas.core.frame.DataFrame'>
75


### Put data into data frame across all items in database

In [16]:
i = 0
while i < len(results):
    # each items in results to iterate and convert into dataframe
    page_properties = results[i].get('properties')
    page_keys = page_properties.keys()
    page_values = page_properties.values()

    # get page URL
    page_properties['page_url'] = results[i].get('url')

    # create dataframe
    df_line = pd.DataFrame(list([page_values]), columns=list(page_keys))
    df = df._append(df_line)
    i += 1
df = df.reset_index()
df.shape
df

,index,Category,Summary,Current page,Total pages,Date started,Progress,Author,Status,Title,page_url,cover,Date finished
0,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'QK`M', 'type': 'number', 'number': 20}","{'id': '\?}w', 'type': 'number', 'number': 920}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Moby-Dick-8ae970fcc5034b...,NaN,NaN
1,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 250}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Winterbienen-36a8bcd366d...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start..."
2,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 468}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Mythos-eb60075d9db74e36b...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start..."
3,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 320}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/ber-Menschen-a044186a860...,NaN,"{'id': 'flfJ', 'type': 'date', 'date': {'start..."
4,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 208}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Zur-See-b30e1dd1d3c64ce4...,NaN,"{'id': 'flfJ', 'type': 'date', 'date': {'start..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 661}",NaN,"{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ali-f9338061df794fd3b908...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...",NaN
71,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 195}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Anaximander-3f8a812a9b9a...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start..."
72,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 201}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id'

## Cleaning data column by column

In [17]:
# make copy of our dataframe
Notion_data = df
print(f'Shape of dataframe:{Notion_data.shape}')
print(Notion_data.columns)

Shape of dataframe:(75, 13)
Index(['index', 'Category', 'Summary', 'Current page', 'Total pages',
       'Date started', 'Progress', 'Author', 'Status', 'Title', 'page_url',
       'cover', 'Date finished'],
      dtype='object')


#### Link data

In [18]:
def clean_link(value):
    try:
        if type(value) is dict:
            value = (value.get('url'))
    except:
        value = 'error'
    return value

#### Name Data

In [19]:
# df['Name'].iloc[1].get('title')[0].get('plain_text')
def clean_name(value):
    try:
        if len(value.get('title')) != 0:
            ret_ = value.get('title')[0].get('plain_text')
        else:
            ret_ = None  # value.get('title')
    except:
        ret_ = 'error'
    return ret_

#### Author data

In [82]:
def clean_author(row):
    try:
        ret_ = row["rich_text"][0]["plain_text"]
    except IndexError:
        ret_ = None
    return ret_

#### Status data

In [58]:
# not being used
def clean_status(value):
    if type(value) is dict:
        ret_ = value.get('select').get('name')
    else:
        ret_ = value
    return ret_

#### Update dataframe using Lambda Functions

In [83]:
Notion_data['_Name'] = Notion_data['Title'].apply(lambda x: clean_name(x))
# apply on axis 1
Notion_data['_Author'] = Notion_data['Author'].apply(lambda x: clean_author(x))
Notion_data['page_id'] = Notion_data['page_url'].apply(lambda x: x[-32:])

### Final dataframe with Notion Data

In [84]:
Notion_data

,index,Category,Summary,Current page,Total pages,Date started,Progress,Author,Status,Title,page_url,cover,Date finished,_Name,_Author,page_id
0,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'QK`M', 'type': 'number', 'number': 20}","{'id': '\?}w', 'type': 'number', 'number': 920}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Moby-Dick-8ae970fcc5034b...,NaN,NaN,Moby Dick,Herman Melville,8ae970fcc5034b6ca551eee7e434b2b4
1,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 250}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Winterbienen-36a8bcd366d...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start...",Winterbienen,Norbert Scheuer,36a8bcd366d64786b768eb2509ff3e6f
2,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 468}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Mythos-eb60075d9db74e36b...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start...",Mythos,Stephen Fry,eb60075d9db74e36ba530479bb452c36
3,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 320}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/ber-Menschen-a044186a860...,NaN,"{'id': 'flfJ', 'type': 'date', 'date': {'start...",Über Menschen,Juli Zeh,a044186a86014c3b89fb198155814ada
4,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 208}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Zur-See-b30e1dd1d3c64ce4...,NaN,"{'id': 'flfJ', 'type': 'date', 'date': {'start...",Zur See,Dörte Hansen,b30e1dd1d3c64ce48d768bde69d3c2a8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 661}",NaN,"{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ali-f9338061df794fd3b908...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...",NaN,Ali,Jonathan Eig,f9338061df794fd3b9086bad63615b78
71,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 195}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Anaximander-3f8a812a9b9a...,"{'id': 'U

### List of page IDs for all items in database

In [85]:
Notion_data['page_id']

0     8ae970fcc5034b6ca551eee7e434b2b4
1     36a8bcd366d64786b768eb2509ff3e6f
2     eb60075d9db74e36ba530479bb452c36
3     a044186a86014c3b89fb198155814ada
4     b30e1dd1d3c64ce48d768bde69d3c2a8
                    ...               
70    f9338061df794fd3b9086bad63615b78
71    3f8a812a9b9a40afac54d856189e7d7c
72    fcfb16d9e44e48768fc04d1f995ffd3a
73    7a403c575b4c4191b47d775e554c489e
74    7cf9cf2022604b6a96e4294a83faf7d7
Name: page_id, Length: 75, dtype: object

## Get google books API data for the Books in my Reading List

### Create a list of books for google api search

In [86]:
my_list = list(filter(None, Notion_data['_Name']))

In [87]:
my_list_no_nan = []
for i in my_list:
    try:
        if pd.isnull(i) != True:
            # print('is null')
            my_list_no_nan += [i]
        # else:
            # print('null')
            # my_list_no_nan += [i]
    except:
        print('except')

print(len(my_list_no_nan))
print(my_list_no_nan)

74
['Moby Dick', 'Winterbienen', 'Mythos', 'Über Menschen', 'Zur See', 'The Illustrated A Brief History of Time', 'To Kill a Mockingbird', 'The Innovators', 'The Code Breaker', 'Sportbootführerschein Binnen Segel/Motor', 'The Children of Hurin', 'Sei du selbst', 'Prima facie', 'SPQR', 'Praxisbuch Usability & UX', 'Offene See', 'Noch wach?', 'Physik', 'Nachrichten aus Mittelerde', 'Napoleon und seine Zeit', 'Montecristo', 'My Own Words', 'Mutabor', 'Monschau', 'MONEYMAKERS', 'Madame Bovary', 'Lichtspiel', 'MANIAC', 'Leonardo da Vinci', 'Krieg und Frieden', 'Klara und die Sonne', 'King: A Life', 'Ich bin Circe', 'Humankind', 'J. Robert Oppenheimer', 'How to Avoid a Climate Disaster', 'How Democracies Die', 'Homer', 'Hamster im hinteren Stromgebiet', 'Helden', 'Hast du uns endlich gefunden', 'Hamlet', 'Gefährlicher Glaube', 'Ensel und Krete', 'Ein verheißenes Land', 'Elon Musk', 'Einstein', 'Dune', 'Diese Wahrheiten', 'Die weite Wildnis', 'Die große Fotoschule', 'Die Verteidigung', 'Die F

### Run list through goole api to get data about the books

In [89]:
# my_reading_list = ['Never Split The Difference', "Seeing what others don't", '48 Laws of Power']
# search_terms = 'Never Split The Difference'

info = pd.DataFrame()
for books in my_list_no_nan:
    print(books)
    info = info._append(google_book_search(books))

Moby Dick


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/var/folders/70/_kzl44wj5hv4tkqjfdkcmjdr0000gn/T/ipykernel_66453/3126177984.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_df['search_term']=search_terms
/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification

Winterbienen


/var/folders/70/_kzl44wj5hv4tkqjfdkcmjdr0000gn/T/ipykernel_66453/3126177984.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_df['search_term']=search_terms
/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Mythos


...

/var/folders/70/_kzl44wj5hv4tkqjfdkcmjdr0000gn/T/ipykernel_66453/3126177984.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_df['search_term']=search_terms
/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Annette, ein Heldinnenepos


/var/folders/70/_kzl44wj5hv4tkqjfdkcmjdr0000gn/T/ipykernel_66453/3126177984.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_df['search_term']=search_terms
/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


22 Bahnen


/var/folders/70/_kzl44wj5hv4tkqjfdkcmjdr0000gn/T/ipykernel_66453/3126177984.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_df['search_term']=search_terms


### Data we received from Google Books API

In [90]:
info.columns

Index(['kind', 'id', 'etag', 'selfLink', 'volumeInfo.title',
       'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
       'volumeInfo.publishedDate', 'volumeInfo.description',
       'volumeInfo.industryIdentifiers', 'volumeInfo.readingModes.text',
       'volumeInfo.readingModes.image', 'volumeInfo.pageCount',
       'volumeInfo.printType', 'volumeInfo.categories',
       'volumeInfo.maturityRating', 'volumeInfo.allowAnonLogging',
       'volumeInfo.contentVersion',
       'volumeInfo.panelizationSummary.containsEpubBubbles',
       'volumeInfo.panelizationSummary.containsImageBubbles',
       'volumeInfo.imageLinks.smallThumbnail',
       'volumeInfo.imageLinks.thumbnail', 'volumeInfo.language',
       'volumeInfo.previewLink', 'volumeInfo.infoLink',
       'volumeInfo.canonicalVolumeLink', 'saleInfo.country',
       'saleInfo.saleability', 'saleInfo.isEbook', 'saleInfo.listPrice.amount',
       'saleInfo.listPrice.currencyCode', 'saleInfo.retailPrice.amount',
 

In [91]:
print(info.shape)
google_data = info[['selfLink', 'volumeInfo.title',
                    'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
                    'volumeInfo.publishedDate', 'volumeInfo.description', 'volumeInfo.pageCount', 'volumeInfo.categories',
                    'volumeInfo.imageLinks.smallThumbnail', 'volumeInfo.imageLinks.thumbnail', 'saleInfo.country', 'saleInfo.retailPrice.amount',
                    'saleInfo.retailPrice.currencyCode', 'search_term'
                    ]]

(74, 61)


### Reset index on google_data dataframe

In [92]:
google_data = (google_data).reset_index()

In [93]:
google_data['volumeInfo.title']

0                      Moby Dick
1                   Winterbienen
2                         Mythos
3                  Über Menschen
4                        Zur See
                 ...            
69         Abriss der Astronomie
70                      アリ (Ali)
71    Anaximander und Anaximenes
72    Annette, ein Heldinnenepos
73                     22 Bahnen
Name: volumeInfo.title, Length: 74, dtype: object

## Dataset from Notion

In [94]:
Notion_data

,index,Category,Summary,Current page,Total pages,Date started,Progress,Author,Status,Title,page_url,cover,Date finished,_Name,_Author,page_id
0,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'QK`M', 'type': 'number', 'number': 20}","{'id': '\?}w', 'type': 'number', 'number': 920}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Moby-Dick-8ae970fcc5034b...,NaN,NaN,Moby Dick,Herman Melville,8ae970fcc5034b6ca551eee7e434b2b4
1,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 250}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Winterbienen-36a8bcd366d...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start...",Winterbienen,Norbert Scheuer,36a8bcd366d64786b768eb2509ff3e6f
2,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 468}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Mythos-eb60075d9db74e36b...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...","{'id': 'flfJ', 'type': 'date', 'date': {'start...",Mythos,Stephen Fry,eb60075d9db74e36ba530479bb452c36
3,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 320}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/ber-Menschen-a044186a860...,NaN,"{'id': 'flfJ', 'type': 'date', 'date': {'start...",Über Menschen,Juli Zeh,a044186a86014c3b89fb198155814ada
4,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 208}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Zur-See-b30e1dd1d3c64ce4...,NaN,"{'id': 'flfJ', 'type': 'date', 'date': {'start...",Zur See,Dörte Hansen,b30e1dd1d3c64ce48d768bde69d3c2a8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 661}",NaN,"{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ali-f9338061df794fd3b908...,"{'id': 'Urz>', 'type': 'url', 'url': 'http://b...",NaN,Ali,Jonathan Eig,f9338061df794fd3b9086bad63615b78
71,0,NaN,"{'id': '?\25', 'type': 'rich_text', 'rich_text...",NaN,"{'id': '\?}w', 'type': 'number', 'number': 195}","{'id': 'evL[', 'type': 'date', 'date': {'start...","{'id': 'nviz', 'type': 'formula', 'formula': {...","{'id': 'qNw_', 'type': 'rich_text', 'rich_text...","{'id': 'qRc\', 'type': 'status', 'status': {'i...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Anaximander-3f8a812a9b9a...,"{'id': 'U

## Dataset from Google API

In [95]:
google_data

,index,selfLink,volumeInfo.title,volumeInfo.subtitle,volumeInfo.authors,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,volumeInfo.imageLinks.thumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,search_term
0,0,https://www.googleapis.com/books/v1/volumes/Vq...,Moby Dick,Der weiße Wal,[Herman Melville],e-artnow,2016-07-31,"Dieses eBook: ""Moby Dick"" ist mit einem detail...",187.0,[Fiction],http://books.google.com/books/content?id=Vqw2D...,http://books.google.com/books/content?id=Vqw2D...,DE,1.99,EUR,Moby Dick
1,0,https://www.googleapis.com/books/v1/volumes/ui...,Winterbienen,Roman,[Norbert Scheuer],C.H.Beck,2019-07-18,Januar 1944: Während über der Eifel britische ...,250.0,[Fiction],http://books.google.com/books/content?id=uiiVD...,http://books.google.com/books/content?id=uiiVD...,DE,9.49,EUR,Winterbienen
2,0,https://www.googleapis.com/books/v1/volumes/BY...,Mythos,Zur Geschichte einer Denkgewohnheit,[Gerhart von Graevenitz],Springer-Verlag,2016-12-17,Was ist Mythos? Der Autor charakterisiert den ...,368.0,[Literary Criticism],http://books.google.com/books/content?id=BYy8D...,http://books.google.com/books/content?id=BYy8D...,DE,29.99,EUR,Mythos
3,0,https://www.googleapis.com/books/v1/volumes/nG...,Über Menschen,Roman,[Juli Zeh],Luchterhand Literaturverlag,2021-03-22,Dora ist mit ihrer kleinen Hündin aufs Land ge...,351.0,[Fiction],http://books.google.com/books/content?id=nGIGE...,http://books.google.com/books/content?id=nGIGE...,DE,10.99,EUR,Über Menschen
4,0,https://www.googleapis.com/books/v1/volumes/PK...,Zur See,NaN,[D. Von Henk],BoD – Books on Demand,2011-08,NaN,458.0,[Transportation],http://books.google.com/books/content?id=PKbla...,http://books.google.com/books/content?id=PKbla...,DE,NaN,NaN,Zur See
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0,https://www.googleapis.com/books/v1/volumes/lq...,Abriss der Astronomie,NaN,[Hans-Heinrich Voigt],John Wiley & Sons,2012-05-07,Die Astronomie untersucht die Eigenschaften de...,1170.0,[Science],http://books.google.com/books/content?id=lqEtB...,http://books.google.com/books/content?id=lqEtB...,DE,NaN,NaN,Astronomie
70,0,https://www.googleapis.com/books/v1/volumes/tO...,アリ (Ali),"Ali Auf Japanisch 6 X 9 (15,24 Cm X 22,86 Cm) ...",[Ali Geschenkidee],NaN,2020-08-28,Ein liniertes Notizbuch in Standartgröße. Auf ...,100.0,NaN,NaN,NaN,DE,NaN,NaN,Ali
71,0,https://www.googleapis.com/books/v1/volumes/rR...,Anaximander und Anaximenes,NaN,NaN,Walter de Gruyter,2011-12-23,The edition of the works of the three sixth-ce...,540.0,[History],http://books.google.com/books/content?id=rR3bI...,http://books.google.com/books/content?id=rR3bI...,DE,144.95,EUR,Anaximander
72,0,https://www.googleapis.com/books/v1/volumes/1n...,"Annette, ein Heldinnenepos",NaN,[Anne Weber],Matthes & Seitz Berlin Verlag,2020-09-02,Mit dem Deutschen Buchpreis 2020 ausgezeichnet...,161.0,[Poetry],http://books.google.com/books/content?id=1nf4D...,http://books.google.com/books/content?id=1nf4D...,DE,8.99,EUR,"Annette, ein Heldinnenepos"


In [96]:
look_up = pd.merge(Notion_data, google_data, left_on=[
                   '_Name'], right_on=['search_term'], how='left')

# Update notion with google data
https://developers.notion.com/reference/page#property-value-object lists the properties and how to format them

In [97]:
# Update a property on a page based on property type
def update_page(page_id, property_name, property_type, property_value):
    url = f"https://api.notion.com/v1/pages/{page_id}"

    if property_type == 'date':
        property_payload = {
            "start": property_value
        }
    elif property_type == 'url':
        property_payload = property_value
    elif property_type == 'number':
        property_payload = property_value
    elif property_type == 'rich_text':
        property_payload = [{
            "type": "text",
            "text": {
                "content": property_value
            }
        }]
    elif property_type == 'select':
        property_payload = {
            "name": property_value
        }

    payload = json.dumps({
        "properties": {
            property_name: {
                property_type: property_payload
            }
        }
    })
    # print(payload)
    headers = {
        'Content-Type': 'application/json',
        'Notion-Version': '2021-05-13',
        'Authorization': f'Bearer {secret_key}'
    }

    response = requests.request(
        "PATCH", url, headers=headers, data=payload, verify=False)
    print(response.status_code)
    # print(response.json)
    return response.json

In [98]:
def update_properties(property_name, property_type, data_column):
    page_id = look_up['page_id'][i]
    property_value = look_up[data_column][i]
    update_page(page_id, property_name, property_type, property_value)

### Update Publishing Dates using Google data

In [99]:
# Update publishing dates
property_name = 'Publishing/Release Date'
property_type = "date"
data_column = 'volumeInfo.publishedDate'

for i in look_up['page_id'].index:
    # if math.isnan(look_up['Publishing/Release Date'][i]):
    update_properties(property_name, property_type, data_column)

/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


### Update Links to Google Books Links

In [101]:
# Update Links
property_name = 'Link'
property_type = "url"
data_column = 'selfLink'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

200


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


### Update Publisher Names

In [100]:
# Update Publishers
property_name = 'Publisher'
property_type = "select"
data_column = 'volumeInfo.publisher'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


### Update number of pages

In [102]:
# Update Number of pages
property_name = 'Pages'
property_type = "number"
data_column = 'volumeInfo.pageCount'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


### Update Summaries

In [103]:
# Update Number of Summary
property_name = 'Summary'
property_type = "rich_text"
data_column = 'volumeInfo.description'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

200


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


### Update Icons for pages to thumbnail images from Google Boooks

In [104]:
# Update icons for pages
data_column = 'volumeInfo.imageLinks.smallThumbnail'


def update_page_icon(i, data_column, icon_or_cover):

    page_id = look_up['page_id'][i]
    property_value = look_up[data_column][i]

    url = f"https://api.notion.com/v1/pages/{page_id}"

    payload = json.dumps({icon_or_cover: {
        "type": "external",
        "external": {
                "url": property_value
        }}})
    print(payload)

    headers = {
        'Content-Type': 'application/json',
        'Notion-Version': '2021-05-13',
        'Authorization': f'Bearer {secret_key}'
    }

    response = requests.request(
        "PATCH", url, headers=headers, data=payload, verify=False)
    print(response.status_code)
    print(response.json)

In [105]:
# Update icons for pages
for i in look_up['page_id'].index:
    update_page_icon(i, data_column, 'icon')

{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=Vqw2DQAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=uiiVDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=BYy8DQAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


...

200
<bound method Response.json of <Response [200]>>
{"icon": {"type": "external", "external": {"url": NaN}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400
<bound method Response.json of <Response [400]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=dN2aEAAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>


In [106]:
# Update cover for pages
for i in look_up['page_id'].index:
    update_page_icon(i, data_column, 'cover')

{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=Vqw2DQAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=uiiVDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=BYy8DQAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


...

200
<bound method Response.json of <Response [200]>>
{"cover": {"type": "external", "external": {"url": NaN}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


400
<bound method Response.json of <Response [400]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=dN2aEAAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/christian/Projects/Notion-Bookshelf/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>


This should have your Reading list notion page updated with Icons, Covers along with Publisher names, published dates, page counts, etc. You can toggle to Gallery view and you have your very own gorgeous Notion books shelf.
<img src="https://i.ibb.co/dkqj9Nk/Screen-Shot-2021-12-30-at-12-18-25-PM.png" alt="Screen-Shot-2021-12-30-at-12-18-25-PM" border="0">
## Useful Links

- [https://developers.notion.com/](https://developers.notion.com/)
- [https://towardsdatascience.com/productivity-tracking-with-the-notion-api-and-python-f5f866fe11d8](https://towardsdatascience.com/productivity-tracking-with-the-notion-api-and-python-f5f866fe11d8)

## Sandbox

In [322]:
look_up[3:5]

,index_x,Type,Summary,Author,Created Time,Topic,Name,page_url,Score /5,Status,...,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,search_term
3,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ask-your-developer-ee339...,NaN,"{'id': '`zz5', 'type': 'select', 'select': {'i...",...,HarperCollins,2021-01-12,"Jeff Lawson, software developer turned CEO of ...",304.0,[Business & Economics],http://books.google.com/books/content?id=lT3LD...,US,1.99,USD,Ask your developer
4,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/48-laws-of-power-9ae1a83...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '`zz5', 'type': 'select', 'select': {'i...",...,NaN,1998-07-01,"Cunning, instructive, and amoral, this controv...",452.0,NaN,http://books.google.com/books/content?id=k2meP...,US,NaN,NaN,48 laws of power


In [292]:
look_up.columns

Index(['index_x', 'Type', 'Summary', 'Author', 'Created Time', 'Topic', 'Name',
       'page_url', 'Score /5', 'Status', 'Completed date', 'Publisher',
       'Publishing/Release Date', 'Pages', 'Link', '_Name', '_Link', '_Author',
       'page_id', 'index_y', 'selfLink', 'volumeInfo.title',
       'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
       'volumeInfo.publishedDate', 'volumeInfo.description',
       'volumeInfo.pageCount', 'volumeInfo.categories',
       'volumeInfo.imageLinks.smallThumbnail', 'saleInfo.country',
       'saleInfo.retailPrice.amount', 'saleInfo.retailPrice.currencyCode',
       'search_term'],
      dtype='object')

In [305]:
page_id = look_up['page_id'][5]
print(page_id)
property_name = 'Status'
property_type = "select"
data_source = 'volumeInfo.description'
property_value = 'Testing summary update'
url = f"https://api.notion.com/v1/pages/{page_id}"

if property_type == 'date':
    property_payload = {
        "start": property_value
    }
elif property_type == 'url':
    property_payload = property_value
elif property_type == 'number':
    property_payload = property_value
elif property_type == 'rich_text':
    property_payload = [{
        "type": "text",
        "text": {
            "content": property_value
        }
    }]

payload = json.dumps({
    "properties": {
        property_name: {
            property_type: property_payload
        }
    }
})
print(payload)

headers = {
    'Content-Type': 'application/json',
    'Notion-Version': '2021-05-13',
    'Authorization': f'Bearer {secret_key}'
}

response = requests.request(
    "PATCH", url, headers=headers, data=payload, verify=False)
print(response.status_code)
print(response.json)

083af95563db484fbb7a26f7c5e76aaa
[{'type': 'text', 'text': {'content': 'Testing summary update'}}]
{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": "Testing summary update"}}]}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>


In [331]:
page_id = look_up['page_id'][5]
print(page_id)
print(look_up.iloc[5]['volumeInfo.imageLinks.smallThumbnail'])
print(look_up.iloc[5]['page_url'])

083af95563db484fbb7a26f7c5e76aaa
http://books.google.com/books/content?id=ya7qDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
https://www.notion.so/Spark-How-to-lead-yourself-and-others-to-greatness-083af95563db484fbb7a26f7c5e76aaa
